In [77]:
import pandas as pd
import requests, csv, time, os
from typing import Optional, Dict
from functools import lru_cache  # CACHE

In [78]:
API_ENDPOINT = "https://www.wikidata.org/w/api.php"  # API
HEADERS = {
    "User-Agent": "NFDI4Microbiota-QS-Generator/2.0 (info@example.com)",
}

MAX_RETRIES = 3    # API
BACKOFF_SECS = 3   # API – MediaWiki ist großzügiger, kleineres Back‑off

In [79]:
def _api_get(params: Dict) -> Dict:  # API
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = requests.get(API_ENDPOINT, params=params, headers=HEADERS, timeout=25)
            r.raise_for_status()
            return r.json()
        except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError):
            wait = BACKOFF_SECS * attempt
            print(f"[warn] API timeout – Versuch {attempt}/{MAX_RETRIES}, warte {wait}s …")
            time.sleep(wait)
        except requests.exceptions.HTTPError as e:
            print(f"[error] API HTTP {e.response.status_code}: {e.response.reason}")
            break
    return {}

In [80]:
@lru_cache(maxsize=None)  # API/cache
def find_qid_by_orcid(orcid: str) -> Optional[str]:
    """Find Q‑ID via ORCID using Cirrus 'haswbstatement' search."""
    if not orcid:
        return None
    query = f'haswbstatement:P496="{orcid}"'
    data = _api_get({
        "action": "query", "list": "search", "srsearch": query,
        "srlimit": 1, "format": "json"})
    try:
        return data["query"]["search"][0]["title"]
    except (KeyError, IndexError):
        return None

In [81]:
@lru_cache(maxsize=None)  # API/cache
def find_qid_by_name(name: str, lang: str = "en") -> Optional[str]:
    if not name:
        return None
    data = _api_get({
        "action": "wbsearchentities", "search": name, "language": lang,
        "type": "item", "limit": 1, "format": "json"})
    try:
        return data["search"][0]["id"]
    except (KeyError, IndexError):
        return None

In [82]:
inst_cache: Dict[str, Optional[str]] = {}

def find_qid_by_institution_label(label: str) -> Optional[str]:  # API
    if not label:
        return None
    if label in inst_cache:
        return inst_cache[label]

    for lang in ("en", "de"):
        data = _api_get({
            "action": "wbsearchentities", "search": label, "language": lang,
            "type": "item", "limit": 1, "format": "json"})
        if data.get("search"):
            qid = data["search"][0]["id"]
            inst_cache[label] = qid
            if lang == "de":
                print(f"[info] Institution '{label}' über deutsches Label gefunden → {qid}")
            return qid

    inst_cache[label] = None
    return None

In [83]:
def file_to_qs(infile: str, outfile: str) -> None:
    ext = os.path.splitext(infile)[1].lower()
    df = pd.read_excel(infile) if ext in {".xlsx", ".xls"} else pd.read_csv(infile)

    required = {"Name", "Institution", "ORCID", "ORCID-Link"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Fehlende Spalten: {', '.join(sorted(missing))}")

    rows = []
    processed = set()

    for _, r in df.iterrows():
        name = str(r["Name"]).strip()
        orcid = str(r["ORCID"]).strip() if pd.notna(r["ORCID"]) else ""
        key = (name.lower(), orcid)
        if key in processed:
            continue
        processed.add(key)

        inst_label = str(r["Institution"]).strip()
        url = r["ORCID-Link"] if pd.notna(r["ORCID-Link"]) else ""

        qid = find_qid_by_orcid(orcid) or find_qid_by_name(name)
        if qid:
            print(f"[skip] {name} existiert bereits als {qid}")
            continue

        inst_qid = find_qid_by_institution_label(inst_label)
        if not inst_qid:
            print(f"[warn] Institution '{inst_label}' nicht gefunden ⇒ übersprungen")
            continue

        rows.append({
            "qid": "",
            "Len": name,
            "P31": "Q5",
            "P496": orcid,
            "S854": url,
            "P108": inst_qid,
        })
        time.sleep(0.1)  # kleine Pause, um API freundlich zu bleiben

    if not rows:
        print("Keine neuen Items – nichts exportiert.")
        return

    field_order = ["qid", "Len", "P31", "P496", "S854", "P108"]
    with open(outfile, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=field_order)
        writer.writeheader()
        writer.writerows(rows)
    print(f"✓ {len(rows)} QuickStatements-Zeilen → {outfile}")

In [84]:
csv_input_path = "import/input_with_orcid.csv"
csv_output_path = "import/quickstatements.csv"

file_to_qs(csv_input_path, csv_output_path)

[skip] Alexander Sczyrba existiert bereits als Q30420936
[skip] Jens Stoye existiert bereits als Q89498719
[skip] Michael Beckstette existiert bereits als Q114411617
[skip] Liren Huang existiert bereits als Q114780829
[skip] Sebastian Jünemann existiert bereits als Q56948964
[skip] Kassian Kobert existiert bereits als Q133094637
[skip] Anandhi Iyappan existiert bereits als Q59196905
[skip] Peer Bork existiert bereits als Q7160367
[skip] Sarah Schulz existiert bereits als Q65162179
[skip] Daniel Podlesny existiert bereits als Q133331882
[skip] Manja Marz existiert bereits als Q87730329
[skip] Winfried Göttsch existiert bereits als Q44200631
[skip] Anderson Santos existiert bereits als Q39510481
[skip] Ulisses Nunes da Rocha existiert bereits als Q47007256
[skip] Martin Bole existiert bereits als Q102304978
[skip] Adrian Fritz existiert bereits als Q133333363
[skip] Alice McHardy existiert bereits als Q2646932
[skip] Mattea Müller existiert bereits als Q56957915
[skip] Fernando Meyer exi